In [8]:
# Make sure that you have the following installed on your machine to view interactive plots in jupyter lab
# https://github.com/matplotlib/ipympl#installation
# run 
#pip install ipympl
#jupyter labextension install @jupyter-widgets/jupyterlab-manager
#jupyter lab build

In [9]:
%matplotlib inline

In [10]:
import math as m
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import writers

In [11]:
def calculate_weights(rows, cols, R):

    # dummy array to easily loop over coordinates
    agents = np.zeros((rows, cols))
    
    # for each agent store the distances to all other nodes
    distances = np.zeros((rows, cols, rows, cols))

    for i1, agent1 in np.ndenumerate(agents):
        x_coord_self = i1[1]
        y_coord_self = i1[0]

        for i2, agent2 in np.ndenumerate(agents):
            x_coord_1 = i2[1]
            x_coord_2 = i2[1]-cols
            y_coord = i2[0]

            distance_1 = np.sqrt((x_coord_1 - x_coord_self)**2 + (y_coord - y_coord_self)**2)
            distance_2 = np.sqrt((x_coord_2 - x_coord_self)**2 + (y_coord - y_coord_self)**2)
            distances[y_coord_self][x_coord_self][y_coord][x_coord_1] = np.min([distance_1,distance_2])
    
    # print(distances[2][1])


    # for each agent store the angle to all of its neighbors
    angles = np.zeros((rows, cols, rows, cols))

    for i1, agent1 in np.ndenumerate(agents):
        x_coord_self = i1[1]
        y_coord_self = i1[0]

        for i2, agent2 in np.ndenumerate(agents):
            x_coord_1 = i2[1]
            x_coord_2 = i2[1]-cols
            
            if x_coord_2 <= -cols/2+1:
                x_coord = x_coord_1
            else:
                x_coord = x_coord_2
            y_coord = i2[0]

            vec_1 = (1, 0)
            vec_2 = (x_coord - x_coord_self, y_coord - y_coord_self)

            c = np.dot(vec_1, vec_2)/np.linalg.norm(vec_1)/np.linalg.norm(vec_2) # -> cosine of the angle
            angle = np.arccos(np.clip(c, -1, 1)) # if you really want the angle
            
            angles[y_coord_self][x_coord_self][y_coord][x_coord_1] = angle
    
    # print(angles[2][1])


    # for each agent store the impact (weight) that all other nodes have on it.
    weights = np.zeros((rows, cols, rows, cols))

    for i1, agent1 in np.ndenumerate(agents):
        x_coord_self = i1[1]
        y_coord_self = i1[0]

        for i2, agent2 in np.ndenumerate(agents):
            x_coord = i2[1]
            y_coord = i2[0]

            distance = distances[y_coord_self][x_coord_self][y_coord][x_coord]
            angle = angles[y_coord_self][x_coord_self][y_coord][x_coord]

            weight = np.exp(- distance / R) * (1 + w + (1 - w) * np.cos(np.pi - angle))

            if distance > 3*R or distance < 0:
                weight = 0

            weights[y_coord_self][x_coord_self][y_coord][x_coord] = weight

        # normalize the weights so that the sum is equal to 1
        # before we can sum up all the weights for each agent, we have to make sure that there is no more nan in the data. nan was always the agent in focus, so it should be 0 that the agent does not influence himself.
        weights[np.isnan(weights)] = 0
        # K is normalizing factor
        K = np.sum(weights[y_coord_self][x_coord_self])
        weights[y_coord_self][x_coord_self] = np.divide(weights[y_coord_self][x_coord_self], K)
        
        
    # print(weights[2][1])
    
    return weights


In [12]:
def set_next_state(agents, weights):
    
    # loop over all agents and calcualte the weighted amount of people in active state that impact this agent
    for i1, agent1 in np.ndenumerate(agents):
        
        x_coord_self = i1[1]
        y_coord_self = i1[0]
        
        weights_agents = weights[y_coord_self, x_coord_self]
    
        state = agent1['state']
        substate = agent1['substate']
        threshold = agent1['threshold']
        
        if state == 1:
            if substate < 2:
                agent1['substate'] += 1
            else:
                agent1['next_state'] = 2
                agent1['substate'] = 0
        
        elif state == 2:
            if substate < 2:
                agent1['substate'] += 1
            else:
                agent1['next_state'] = 0
                agent1['substate'] = 0
                
        else:
            # current state: 0 (inactive)
            # calculate the probability that he becomes active
            active_agents_weight = 0
            
            for i2, agent2 in np.ndenumerate(agents):
                x_coord = i2[1]
                y_coord = i2[0]
                
                # sum over all the agents in the radius that have an influence on the agent in question
                if agent2['state'] == 1:
                    weight = weights_agents[y_coord, x_coord]
                    active_agents_weight += weight
        
            
            if active_agents_weight > threshold:
                # update agent to be in active state
                agent1['next_state'] = 1
                agent1['substate'] = 0

                
def update_state(agents):
    for i, agent in np.ndenumerate(agents):
        agent['state'] = agent['next_state']
        
        
def get_na(agents,rows, cols):
    res_aggregate = np.zeros(cols)
    res_state_only = np.zeros((rows, cols))

    # sum all cols up
    for index, agent in np.ndenumerate(agents):
        y_coord = index[0]
        x_coord = index[1]
        state = agent['state']

        res_state_only[y_coord][x_coord] = state

        if state == 1:
            res_aggregate[x_coord] += 1
            
    return res_aggregate, res_state_only


def get_a_max(res_aggregate):
    return np.amax(res_aggregate, axis=1)
        

def run_simulation(agents, weights, rows, cols, threshold, size_triggering_group, time_steps):
    
    # N_a(t)
    res_aggregate_total = []
    res_state_only_total = []

    # initialise thresholds and states
    for i in range(rows):
        for j in range(cols):
            agents[i][j]['state'] = 0
            agents[i][j]['substate'] = 0
            agents[i][j]['next_state'] = 0
            agents[i][j]['threshold'] = np.random.uniform(threshold - 0.05, threshold + 0.05)

    agents_in_row = size_triggering_group[0]
    agents_in_col = size_triggering_group[1]
    
    for i in range(agents_in_row):
        for j in range(agents_in_col):
            agents[i + (int((rows - agents_in_col) / 2))][j+3]['state'] = 1
            agents[i + (int((rows - agents_in_col) / 2))][j+3]['next_state'] = 1
    
    res_aggregate, res_state_only = get_na(agents, rows, cols)
    
    res_aggregate_total.append(res_aggregate)
    res_state_only_total.append(res_state_only)
    
    for i in range(time_steps):
        res_aggregate = np.zeros(cols)
        res_state_only = np.zeros((rows, cols))
        
        # sum all cols up
        for index, agent in np.ndenumerate(agents):
            y_coord = index[0]
            x_coord = index[1]
            state = agent['state']
            
            res_state_only[y_coord][x_coord] = state
            
            if state == 1:
                res_aggregate[x_coord] += 1
                
        set_next_state(agents, weights)
        
        # update all agents synchronously
        update_state(agents)
        
        res_aggregate, res_state_only = get_na(agents, rows, cols)
        
        res_aggregate_total.append(res_aggregate)
        res_state_only_total.append(res_state_only)
        
    return res_state_only_total, res_aggregate_total
    

In [ ]:
rows = 25
cols = 50

R = 2
# factor of how much more people to the right of a person have an influence compared to the people on the left
w = 0.25

# weights can be pre calculated as they won't change during simulation
weights = calculate_weights(rows, cols, R)

/home/dario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
# threshold for each agent to become active, will be randomly chosen in the interval +-0.05
thresholds = [.25, .26, .27, .28, .29, .3, .31, .32]
# fraction of people on the two most left columns that intitiate the wave
size_triggering_groups = [(1,2), (2,2), (2,3), (3,3), (3,4), (4,4)]

time_steps = 100

fig = plt.figure()
ax = fig.add_subplot(111)

# animation function.  This is called sequentially
def animate(i, res, colax):
    colax.set_data(res[i])
    return colax

a_max_mean = np.zeros((len(thresholds), len(size_triggering_groups)))
a_max_std = np.zeros((len(thresholds), len(size_triggering_groups)))
aggregated_cols = np.zeros((len(thresholds), len(size_triggering_groups)), dtype=(np.ndarray))

for i1, threshold in enumerate(thresholds):
    
    for i2, size_triggering_group in enumerate(size_triggering_groups):
        
        a_max_temp_mean = []
        a_max_temp_std = []
        aggregated_cols_temp = []

        for i in range(20):
            # each agent has one the following states
            # 0: inactive
            # 1: active
            # 2: refracter
            agents = np.zeros((rows, cols), dtype=[('state', np.int), ('substate', np.int), ('next_state', np.int), ('threshold', np.float)])

            results = run_simulation(agents, weights, rows, cols, threshold, size_triggering_group, time_steps)
            aggregated_cols_temp.append(results[1])
            a_max = get_a_max(results[1])
            a_max_temp_mean.append(np.mean(a_max))
            a_max_temp_std.append(np.std(a_max))
           
            # initialize the plot (each color 0,1,2 is represented once so that the colors are also there for the next frames in the animation)
            initial = np.zeros((rows, cols))
            initial[0][1] = 1
            initial[0][2] = 2
            cax = ax.matshow(initial)
            
            # call the animator
            # blit=True means only re-draw the parts that have changed
            anim = animation.FuncAnimation(fig, animate, fargs=(results[0], cax), frames=len(results[0]))
            #writers.reset_available_writers()

            # On windows/linux make sure to have ffmpeg installed before running this cell
            # https://ffmpeg.org/
            # on a mac you can simply use brew to install: brew install ffmpeg
            anim.save('animations/laOla-' + str(threshold) + '-' + str(size_triggering_group[0] * size_triggering_group[1]) + '-version' + str(i) +'.mp4', writer="ffmpeg", fps=5)
        
        mean_of_a_max_temp_mean = np.mean(a_max_temp_mean)
        mean_of_a_max_temp_std = np.mean(a_max_temp_std)
        print(threshold, size_triggering_group, 'Avg of A(t):', mean_of_a_max_temp_mean, 'Std of A(t)', mean_of_a_max_temp_std)
        a_max_mean[i1][i2] = mean_of_a_max_temp_mean
        a_max_std[i1][i2] = mean_of_a_max_temp_std
        aggregated_cols[i1][i2] = aggregated_cols_temp

In [ ]:
np.save('a_max_mean.npy', a_max_mean)
np.save('a_max_std.npy', a_max_std)
np.save('aggregated_cols', aggregated_cols)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

threshold_plot = [[threshold] * len(size_triggering_groups) for threshold in thresholds]
triggering_groups_plot = [[group[0] * group[1] for group in size_triggering_groups]] * len(thresholds)

fig = plt.figure() 
wf = fig.add_subplot(111, projection='3d') 

wf.plot_wireframe(threshold_plot, triggering_groups_plot, a_max_mean, rstride=1, cstride=1) 
wf.set_xlabel('threshold')
wf.set_ylabel('triggering group')
wf.set_zlabel('Mean of A(t)')

plt.savefig('result_plot_mean.pdf')
plt.show() 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

threshold_plot = [[threshold] * len(size_triggering_groups) for threshold in thresholds]
triggering_groups_plot = [[group[0] * group[1] for group in size_triggering_groups]] * len(thresholds)

fig = plt.figure() 
wf = fig.add_subplot(111, projection='3d') 

wf.plot_wireframe(threshold_plot, triggering_groups_plot, a_max_std, rstride=1, cstride=1)
wf.set_xlabel('threshold')
wf.set_ylabel('triggering group')
wf.set_zlabel('Std of A(t)')

plt.savefig('result_plot_std.pdf')
plt.show() 